In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
import matplotlib.pyplot as plt
import seaborn  as sns

In [ ]:
## 9 algorithms we are going to learn 

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor
from sklearn.neural_network import multilayer_perceptron

In [ ]:
# to find the skewness
from scipy.stats import norm, skew #for some statistics


In [ ]:
# to reduce the skewness or for normalizing
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax



In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.pipeline import make_pipeline

In [ ]:
df_train = pd.read_csv(r"../input/house-prices-dataset/train.csv")
df_test = pd.read_csv(r"../input/house-prices-dataset/test.csv")

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
sns.distplot(df_train['SalePrice']);

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % df_train['SalePrice'].skew())
print("Kurtosis: %f" % df_train['SalePrice'].kurt())

In [ ]:
#df_train = df_train.drop(['Utilities', 'Street', 'PoolQC',], axis=1) # zero corelation 

In [ ]:
train_ID = df_train['Id']
test_ID = df_test['Id']

In [ ]:
# Now drop the  'Id' colum since it's unnecessary for  the prediction process.
df_train.drop(['Id'], axis=1, inplace=True)
df_test.drop(['Id'], axis=1, inplace=True)

In [ ]:
df_train = df_train[df_train.GrLivArea < 4500]
df_train.reset_index(drop=True, inplace=True)

In [ ]:
y = df_train.SalePrice.reset_index(drop=True)
train_features =df_train.drop(['SalePrice'], axis=1)
test_features = df_test

# concatinate the train and the test set as features for tranformation to avoid mismatch
features = pd.concat([train_features, test_features]).reset_index(drop=True)
print('Features size:', features.shape)

In [ ]:
# function for determining the threshold of missing values
def percent_missing(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0, len(df_cols)):
        dict_x.update({df_cols[i]: round(data[df_cols[i]].isnull().mean()*100,2)})
    
    return dict_x

missing = percent_missing(features)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Percent of missing data')
df_miss[0:10]

In [ ]:
# visualising missing values
sns.set_style("white")
f, ax = plt.subplots(figsize=(8, 7))
sns.set_color_codes(palette='deep')
missing = round(df_train.isnull().mean()*100,2)
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar(color="b")
# Tweak the visual presentation
ax.xaxis.grid(False)
ax.set(ylabel="Percent of missing values")
ax.set(xlabel="Features")
ax.set(title="Percent missing data by feature")
sns.despine(trim=True, left=True)

In [ ]:
features = features.drop(['PoolQC','MiscFeature','Alley','Fence','Utilities','Street','FireplaceQu'],axis=1)


In [ ]:
features.dtypes

In [ ]:
### here below 3 columns are numeric has to convrt into class

In [ ]:
features['MSSubClass'].dtype
features['YrSold'].dtype
features['MoSold'].dtype

In [ ]:
features['MSSubClass'] = features['MSSubClass'].apply(str)
features['YrSold'] = features['YrSold'].astype(str)
features['MoSold'] = features['MoSold'].astype(str)

In [ ]:
features['MoSold'].dtype

In [ ]:
def nullValue_Treatment(df,nv_percentage=30):
    df.isna().sum()
    df_nullvalues=(df.isna().sum()/df.shape[0])*100
    #print(df_nullvalues)
    print("                                       ")
    print(df_nullvalues.shape)
    drop_columns=df_nullvalues[df_nullvalues>nv_percentage].index
    retained_columns=df_nullvalues[df_nullvalues<=nv_percentage].index
    len(drop_columns),len(retained_columns),len(df.columns)
    df.drop(drop_columns,axis=1,inplace=True)
    print(df.shape)
    df[retained_columns].shape
    num=df.describe().columns
    cat=[i for i in df.columns if i not in num]
    print(len(num),len(cat))
    for i in cat:
        df[i].fillna(df[i].value_counts().index[0],inplace=True)
    for i in num:
        df[i].fillna(df[i].median(),inplace=True)
    #print(df.isna().sum())
    return [df,num,cat]

In [ ]:
features,num_col , cat_col  = nullValue_Treatment(features)

In [ ]:
features['MSSubClass'] = features.MSSubClass.astype(str)
features['YrSold'] = features.YrSold.astype(str)
features['MoSold'] = features.MoSold.astype(str)


In [ ]:
# Filling the rest of the categorical features
objects = []
for i in features.columns:
    if features[i].dtype == object:
        objects.append(i)
        print(i)
features.update(features[objects].fillna('None'))

In [ ]:
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics2 = []
for i in features.columns:
    if features[i].dtype in numeric_dtypes:
        numerics2.append(i)

In [ ]:
skew_features = features[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index
print("There are {} numerical features with Skew > 0.5 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skew_features.head(10)

In [ ]:
# Normalise skewed features
for i in skew_index:
    features[i] = boxcox1p(features[i], boxcox_normmax(features[i] + 1))

In [ ]:
# Calculating totals before droping less significant columns

#  Adding total sqfootage feature 
features['TotalSF']=features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']
#  Adding total bathrooms feature
features['Total_Bathrooms'] = (features['FullBath'] + (0.5 * features['HalfBath']) +
                               features['BsmtFullBath'] + (0.5 * features['BsmtHalfBath']))
#  Adding total porch sqfootage feature
features['Total_porch_sf'] = (features['OpenPorchSF'] + features['3SsnPorch'] +
                              features['EnclosedPorch'] + features['ScreenPorch'] +
                              features['WoodDeckSF'])

In [ ]:
# Not normaly distributed can not be normalised and has no central tendecy
features = features.drop(['MasVnrArea', 'OpenPorchSF', 'WoodDeckSF', 'BsmtFinSF1','2ndFlrSF'], axis=1)
# Adding new simplified features (1 = present, 0 = not present)
features['haspool'] = features['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
features['hasgarage'] = features['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
features['hasbsmt'] = features['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
features['hasfireplace'] = features['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

print('Features size:', features.shape)

In [ ]:
# Encoding the finalized features
final_features = pd.get_dummies(features).reset_index(drop=True)
print('Features size:', features.shape)
final_features.head()

In [ ]:
# Spliting the data back to train(X,y) and test(X_sub)
X = final_features.iloc[:len(y), :]
X_test = final_features.iloc[len(X):, :]
print('Features size for train(X,y) and test(X_test):')
print('X', X.shape, 'y', y.shape, 'X_test', X_test.shape)

In [ ]:
# Removes outliers 
outliers = [30, 88, 462, 631, 1322]
X = X.drop(X.index[outliers])
y = y.drop(y.index[outliers])

In [ ]:
# Removes colums where the threshold of zero's is (> 99.95), means has only zero values 
overfit = []
for i in X.columns:
    counts = X[i].value_counts()
    zeros = counts.iloc[0]
    if zeros / len(X) * 100 > 99.95:
        overfit.append(i)

overfit = list(overfit)
overfit.append('MSZoning_C (all)')

X = X.drop(overfit, axis=1).copy()
X_test = X_test.drop(overfit, axis=1).copy()


print('X', X.shape, 'y', y.shape, 'X_test', X_test.shape)

In [ ]:
y=np.log(y)

In [ ]:
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)

# model scoring and validation function
def cv_rmse(model, X=X):
    rmse = np.sqrt(-cross_val_score(model, X, y,scoring="neg_mean_squared_error",cv=kfolds))
    return (rmse)

# rmsle scoring function
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
def model_validation(x,y,modelname):   
    lr=modelname()
    lr.fit(x,y)
    lr.predict(x)
    actual=y
    predicted=lr.predict(x)
    error=abs(actual-predicted).sum()
    error
    mae=error/x.shape[0]
    mse=np.square(error).sum()/x.shape[0]
    rmse=np.sqrt(mse)
    mape= (((actual-predicted).sum()/actual.sum())*100)/x.shape[0]
    return [mae,mse,rmse,mape]

In [ ]:
X[:5] y[:5]

In [ ]:
models=[LinearRegression,Lasso,Ridge,ElasticNet,DecisionTreeRegressor]
model_results=[]
for i in models:
    mae,mse,rmse,mape=model_validation(X[:5],y[:5],i)
    model_results.append([i,mae,mse,rmse,mape])

In [ ]:
model_results=pd.DataFrame(model_results)
model_results.columns=["model_name","mae","mse","rmse","mape"]
plt.plot(model_results['mae'],marker='o')

In [ ]:

model_results

In [ ]:
model_results["average"] =model_results.mean(axis=1)

In [ ]:
plt.plot(model_results['mae'],marker='o')

In [ ]:
#models=[LinearRegression,Lasso,Ridge,ElasticNet]
model_results=[]
for i in models:
    mae,mse,rmse,mape=model_validation(X,y,i)
    model_results.append([i,mae,mse,rmse,mape])

In [ ]:
model_results=pd.DataFrame(model_results)
model_results.columns=["model_name","mae","mse","rmse","mape"]
plt.plot(model_results['mae'],marker='o')

In [ ]:
model_results["average"] =model_results.mean(axis=1)
plt.plot(model_results['mae'],marker='o')

In [ ]:
# setup models hyperparameters using a pipline
# The purpose of the pipeline is to assemble several steps that can be cross-validated together, while setting different parameters.
# This is a range of values that the model considers each time in runs a CV
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]


# Kernel Ridge Regression : made robust to outliers
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=alphas_alt, cv=kfolds))
# LASSO Regression : made robust to outliers
lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=1e7, 
                    alphas=alphas2,random_state=42, cv=kfolds))
# Elastic Net Regression : made robust to outliers
elasticnet = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=1e7, 
                         alphas=e_alphas, cv=kfolds, l1_ratio=e_l1ratio))


In [ ]:

# Kernel Ridge Regression : made robust to outliers
ridge = RidgeCV()
# LASSO Regression : made robust to outliers
lasso = LassoCV()

# Elastic Net Regression : made robust to outliers
elasticnet = ElasticNetCV()



In [ ]:
def blend_models(X):
    return ((elastic_model.predict(X)) + (lasso_model.predict(X)) + (ridge_model.predict(X)))/3

In [ ]:

print('Elasticnet')
elastic_model = elasticnet.fit(X, y)


In [ ]:
print('Lasso')
lasso_model = lasso.fit(X, y)

In [ ]:
print('Ridge')
ridge_model = ridge.fit(X, y)

In [ ]:
prediction  = blend_models(X_test)

In [ ]:
len(prediction)

In [ ]:
submission=[]
submission = pd.DataFrame(submission)
submission['Id'] =test_ID

In [ ]:
submission['SalePrice'] =np.expm1(prediction)

In [ ]:
submission.shape

In [ ]:
submission.to_csv(r'submission.csv')


In [ ]:
from IPython.display import FileLink
FileLink(r'submission.csv')

In [ ]:
submission[:5]

In [ ]:
df_test = pd.read_csv(r"../input/house-prices-dataset/test.csv")

In [ ]:
df_test.head()